Import

In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

Locate the credentials file

In [0]:
# list tables in filestore to get name of credentials file
dbutils.fs.ls("/FileStore/tables")

Out[17]: [FileInfo(path='dbfs:/FileStore/tables/0ed442ca38ad_user_table/', name='0ed442ca38ad_user_table/', size=0, modificationTime=1706654849961),
 FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv/', name='authentication_credentials.csv/', size=0, modificationTime=1706654849961),
 FileInfo(path='dbfs:/FileStore/tables/csv_files/', name='csv_files/', size=0, modificationTime=1706654849961),
 FileInfo(path='dbfs:/FileStore/tables/date_details.json', name='date_details.json', size=284166, modificationTime=1703945051000),
 FileInfo(path='dbfs:/FileStore/tables/downloaded_file.json', name='downloaded_file.json', size=48, modificationTime=1703945574000),
 FileInfo(path='dbfs:/FileStore/tables/downloaded_file2.json', name='downloaded_file2.json', size=60, modificationTime=1703954387000),
 FileInfo(path='dbfs:/FileStore/tables/geo_dirty.csv/', name='geo_dirty.csv/', size=0, modificationTime=1706654849961),
 FileInfo(path='dbfs:/FileStore/tables/json_files/', name='json_files/', size=0, modificationTime=1706654849961),
 FileInfo(path='dbfs:/FileStore/tables/maya/', name='maya/', size=0, modificationTime=1706654849961),
 FileInfo(path='dbfs:/FileStore/tables/pin_dirty.csv/', name='pin_dirty.csv/', size=0, modificationTime=1706654849961),
 FileInfo(path='dbfs:/FileStore/tables/single_json_file.json', name='single_json_file.json', size=293, modificationTime=1701857903000),
 FileInfo(path='dbfs:/FileStore/tables/test_credentials.csv', name='test_credentials.csv', size=99, modificationTime=1704283321000),
 FileInfo(path='dbfs:/FileStore/tables/user_dirty.csv/', name='user_dirty.csv/', size=0, modificationTime=1706654849961)]

Read CSV

In [0]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1460384716030528> in <module> 
 1 # Read the CSV file to spark dataframe 
 ----> 2 aws_keys_df = spark . read . format ( "csv" ) \ 
 3 . option ( "header" , "true" ) \ 
 4 . option ( "sep" , "," ) \ 
 5 . load ( "/FileStore/tables/authentication_credentials.csv" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 156 self . options ( ** options ) 
 157 if isinstance ( path , str ) : 
 --> 158 return self . _df ( self . _jreader . load ( path ) ) 
 159 elif path is not None : 
 160 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Unable to infer schema for CSV. It must be specified manually.

Get AWS Access key & Secret key

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0e2a0bfcc015-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0e2a0bfcc015-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Check if Database was mounted

In [0]:
display(dbutils.fs.ls("/mnt/user-0e2a0bfcc015-bucket/topics"))

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-1460384716030541> in <module> 
 ----> 1 display ( dbutils . fs . ls ( "/mnt/user-0e2a0bfcc015-bucket/topics" ) ) 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 387 exc . __context__ = None 
 388 exc . __cause__ = None 
 --> 389 raise exc
 390 
 391 return f_with_exception_handling

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: No such file or directory: s3a://db-76a5715dfb6b72be02702c5187233fcf-s3-root-bucket/nvirginia-prod/1865928197306450/mnt/user-0e2a0bfcc015-bucket/topics
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3390)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3264)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:3203)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.innerListStatus(S3AFileSystem.java:2726)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$listStatus$11(S3AFileSystem.java:2705)
	at shaded.databricks.org.apache.hadoop.fs.s3a.Invoker.once(Invoker.java:118)
	at shaded.databricks.org.apache.hadoop.fs.s3a.Invoker.once(Invoker.java:112)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.listStatus(S3AFileSystem.java:2705)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:95)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:70)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$1(DatabricksFileSystemV2.scala:92)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:444)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:33)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:94)
	at com.databricks.logging.Log4jUsageLoggingShim$.withAttributionContext(Log4jUsageLoggingShim.scala:31)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:205)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:204)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:240)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:225)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:419)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:339)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperationWithResultTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:302)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:510)
	at com.databricks.backend.daemon.data.client.DBFSV2.listStatus(DatabricksFileSystemV2.scala:92)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:164)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$ls$1(DBUtilsCor

DataFrame for pinterest post data

In [0]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location_pin = "/mnt/user-0e2a0bfcc015-bucket/topics/0e2a0bfcc015.pin/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"

# DataFrame for pinterest post data
df_pin = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_pin)

# Display Spark dataframe to check its content
display(df_pin)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1460384716030532> in <module> 
 11 
 12 # DataFrame for pinterest post data 
 ---> 13 df_pin = spark . read . format ( file_type ) \ 
 14 . option ( "inferSchema" , infer_schema ) \ 
 15 . load ( file_location_pin ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 156 self . options ( ** options ) 
 157 if isinstance ( path , str ) : 
 --> 158 return self . _df ( self . _jreader . load ( path ) ) 
 159 elif path is not None : 
 160 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Path does not exist: dbfs:/mnt/user-0e2a0bfcc015-bucket/topics/0e2a0bfcc015.pin/partition=0/*.json

DataFrame for pinterest geo data

In [0]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location_geo = "/mnt/user-0e2a0bfcc015-bucket/topics/0e2a0bfcc015.geo/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"

# DataFrame for pinterest geo data
df_pin = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_geo)

# Display Spark dataframe to check its content
display(df_geo)

DataFrame for pinterest user data

In [0]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location_user = "/mnt/user-0e2a0bfcc015-bucket/topics/0e2a0bfcc015.user/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"

# DataFrame for pinterest user data
df_pin = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_user)

# Display Spark dataframe to check its content
display(df_user)